In [1]:
using RCall
using BenchmarkTools
using CSV
using DataFrames
using GLM
using PenalizedGLMM
using SnpArrays

In [2]:
pkgversion(PenalizedGLMM)

v"0.1.0"

In [3]:
pkgversion(BenchmarkTools)

v"1.6.0"

In [4]:
Threads.nthreads()

1

In [2]:
### data filepaths ###
const datadir = "data/"
const covfile = datadir * "penncath.csv" # includes binary phenotype data
const plinkfile = datadir * "qc_penncath"
const grmfile = datadir * "PennCathGRM.txt.gz" # genetics relatedness matrix

"hpchome/manuscriptcomparison/data/PennCathGRM.txt.gz"

In [3]:
### read in covariate file ###
covdf = CSV.read(covfile, DataFrame)
#fam = CSV.read(famfile, DataFrame, delim = " ", header = false)
#rename!(fam,:Column1 => :FamID)
#covdf = rightjoin(covdf, fam, on = :FamID)

Row,FamID,CAD,sex,age,tg,hdl,ldl
,Int64,Int64,Int64,Int64,String7,String3,String3
1,10002,1,1,60,NA,NA,NA
2,10004,1,2,50,55,23,75
3,10005,1,1,55,105,37,69
4,10007,1,1,52,314,54,108
5,10008,1,1,58,161,40,94
6,10009,1,1,59,171,46,92
7,10010,1,1,54,147,69,109
8,10011,1,2,66,124,47,84
9,10012,1,1,58,60,114,67


In [4]:
# force CAD to be a float (for normal approx)
covdf[:,:CADfloat] = convert(Array{Float64}, covdf[:,:CAD])
CSV.write("penncathfloat.csv", covdf)
const floatcovfile = "penncathfloat.csv"

"penncathfloat.csv"

In [38]:
# Run the benchmark 10 times
# normnull_times = []
# for i in 1:10
#     push!(normnull_times, @belapsed pglmm_null(@formula(CADfloat ~ age + sex), covfile = floatcovfile, grmfile = grmfile, 
#     family = Normal(), link = IdentityLink()))
# end

# Calculate the median and range
# median_time = median(normnull_times)
# min_time = minimum(normnull_times)
# max_time = maximum(normnull_times)

# println("Median time: $median_time seconds")
# println("Minimum: $min_time seconds")
# println("Maximum: $max_time seconds")

Median time: 68.30368197050001 seconds
Minimum: 60.481683923 seconds
Maximum: 75.748191132 seconds


In [6]:
nullmodel_normal = pglmm_null(@formula(CADfloat ~ sex + age), covfile = floatcovfile, grmfile = grmfile, 
   family = Normal(), link = IdentityLink())

(φ = 0.1748913486123, τ = [0.0], α = [2.037336709986514, -0.17025981031668705, -0.02054277167222361], η = [0.6345105993364102, 0.6696785057419594, 0.7372244576975282, 0.7988527727141991, 0.6755961426808573, 0.6550533710086337, 0.7577672293697517, 0.34099415898638163, 0.6755961426808573, 0.32045138731415806  …  0.42316524567527614, 0.5112539693030684, 0.6755961426808573, 0.5112539693030684, 0.6902212774141829, 0.484793560691947, 0.9426521744197642, 0.6550533710086337, 1.2507937495031185, 0.5523395126475156], b = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], converged = true, τV = [1.0e-5 0.0 … 0.0 0.0; 0.0 1.0e-5 … 0.0 0.0; … ; 0.0 0.0 … 1.0e-5 0.0; 0.0 0.0 … 0.0 1.0e-5], y = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0  …  0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0], X = [1.0 1.0 60.0; 1.0 2.0 50.0; … ; 1.0 1.0 30.0; 1.0 1.0 64.0], L = [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], 

In [39]:
@elapsed pglmm_null(@formula(CADfloat ~ sex + age), covfile = floatcovfile, grmfile = grmfile, 
   family = Normal(), link = IdentityLink())

69.827473539

In [38]:
print(nullmodel_normal.converged)

true

In [37]:
# Normal model

# Run the benchmark 10 times
# normfit_times = []
# for i in 1:10
#     push!(normfit_times, @belapsed pglmm(nullmodel_normal, plinkfile))
# end

# Calculate the median and range
# median_time = median(normfit_times)
# min_time = minimum(normfit_times)
# max_time = maximum(normfit_times)

# println("Median time: $median_time seconds")
# println("Minimum: $min_time seconds")
# println("Maximum: $max_time seconds")

Median time: 941.6487381249999 seconds
Minimum: 935.979902024 seconds
Maximum: 981.174038304 seconds


In [ ]:
modelfit_normal = pglmm(nullmodel_normal, plinkfile)

In [ ]:
@belapsed pglmm(nullmodel_normal, plinkfile)